In [6]:
%matplotlib inline
import requests
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs

In [7]:
class Athlete():
    def __init__(self,link):
        self.link = link
        self.request = requests.get(link)
        self.request.encoding='UTF-8'
        self.soup = bs(self.request.text,'lxml')
        self.name = self.soup.find('span',{'class':'athlete-name'}).text
        self.country = self.soup.find('div',{'class':'country-name'}).text
        
        self.characteristics = self.getChars()    
        self.stats = self.getStats()   
        self.events = self.getEvents()
    
    def getChars(self):
        
        labels = ['Stance','First season','Age','Height (cm)','Weight (kg)','Country']
        values=[]
        
        lis = self.soup.find('div',{'class':'new-athlete-bio-stats'}).find_all('li')
        i = 1
        
        for li in lis:
            value = None
            if i == 3:
                value = int(li.find_all('div')[1].text.split('\n')[1].replace(" ",""))
            elif i == 4:
                value = int(li.find_all('div')[1].text.split('\n')[-1].replace(" ","").replace('cm',''))
            elif i == 5:
                value = int(li.find_all('div')[1].text.split('lbs')[-1].replace(" ","").replace('kg',''))
            elif i == 6:
                value = str(self.country)
            else:
                value = li.find_all('div')[1].text
            values.append(value)
            i+=1
         
        d = {}
        for i in range(0, 6):
            d[labels[i]] = values[i]
        chars = pd.Series(d)
        chars.loc['Athlete'] = self.name
        return chars

    def getStats(self):
        
        labels = ["Men's CT 2019 Ranking", "Heat wins", "Avg. heat score", "Rookie year"]
        values = []
        
        lis = self.soup.find('ul',{'class':'new-athlete-stat-bar__stats stat-bar__stats'}).find_all('li')
        for li in lis:
            values.append(li.find('div',{'class':'value'}).text)
       
        d = {}
        for i in range(0, 4):
            if i == 0:
                d[labels[i]] = int(str(values[i]).replace('#',''))
            elif i == 1:
                d[labels[i]] = int(values[i])
            else:
                d[labels[i]] = float(values[i])
        stats = pd.Series(d)
        stats.loc['Athlete'] = self.name
        return stats

    
    def getEvents(self):
        labels1 = ['Event {}: Place'.format(i) for i in range(1, 12)]
        labels2 = ['Event {}: Points'.format(i) for i in range(1, 12)]
        values1 = []
        values2 = []
        
                
        div = self.soup.find('div',{'class':'table-wrap table-wrap--athletes'})
        trs = self.soup.find('div',{'class':'table-wrap table-wrap--athletes'}).find_all('tr')[1:]

        codes = [2908,2909,2912,2913,2916,2917,2919,2920,2923,2924,2927]

        j=0
        matching = []

        for i in range(0, 11):

            matching.append('athlete-event-{}'.format(codes[i]) in str(div))

            if matching[i]:              
                if 'INJ' in trs[j].find_all('td')[1].text:
                    values1.append('INJ')
                else:
                    values1.append(str(trs[j].find_all('td')[1].text))
                values2.append(int(str(trs[j].find_all('td')[2].text).replace(',','')))
                j+=1
            else:
                values1.append("-")
                values2.append(0)
                           
        labels = labels1+labels2
        values = values1+values2
        
        d = {}
        for i in range(0, 22):
            d[labels[i]] = values[i]
        events = pd.Series(d)
        events.loc['Athlete'] = self.name
                           
        return events

### LATER ON, USE THIS FOR ALL ATHLETES:

In [8]:
r = requests.get('https://www.worldsurfleague.com/athletes/tour/mct?year=2019')

In [27]:
r.encoding = 'UTF-8'
soup = bs(r.text,'lxml')
tds = soup.find_all('td',{'class':'athlete-headshot-and-name'})
links = ['http://www.worldsurfleague.com' + td.a['href'] for td in tds]

athletes = []
lchars = []
lstats = []
levents = []

i=0
for link in links[0:36]:     
    ath = Athlete(link)
    athletes.append(ath.name)
    lstats.append(ath.stats)
    levents.append(ath.events)
    lchars.append(ath.characteristics)
    print('Success: {}, {}, ({})'.format(ath.name,ath.country,i))
    i += 1

#ccc = pd.concat([pd.Series(lchars),pd.Series(lstats),pd.Series(levents)],axis=1).T
    
aaa = pd.concat(lstats,axis=1).T
bbb = pd.concat(lchars,axis=1).T
ccc = pd.concat(levents,axis=1).T

ddd = pd.merge(aaa,bbb, how='outer')
eee = pd.merge(ddd,ccc, how='outer')

eee = eee.set_index('Athlete')
fff = pd.merge(ddd,ccc, how='outer')

Success: Italo Ferreira, Brazil, (0)
Success: Gabriel Medina, Brazil, (1)
Success: Jordy Smith, South Africa, (2)
Success: Filipe Toledo, Brazil, (3)
Success: Kolohe Andino, United States, (4)
Success: Kanoa Igarashi, Japan, (5)
Success: John John Florence, Hawaii, (6)
Success: Kelly Slater, United States, (7)
Success: Owen Wright, Australia, (8)
Success: Jeremy Flores, France, (9)
Success: Julian Wilson, Australia, (10)
Success: Seth Moniz, Hawaii, (11)
Success: Michel Bourez, France, (12)
Success: Ryan Callinan, Australia, (13)
Success: Jack Freestone, Australia, (14)
Success: Griffin Colapinto, United States, (15)
Success: Caio Ibelli, Brazil, (16)
Success: Wade Carmichael, Australia, (17)
Success: Adrian Buchan, Australia, (18)
Success: Conner Coffin, United States, (19)
Success: Peterson Crisanto, Brazil, (20)
Success: Yago Dora, Brazil, (21)
Success: Deivid Silva, Brazil, (22)
Success: Willian Cardoso, Brazil, (23)
Success: Jesse Mendes, Brazil, (24)
Success: Michael Rodrigues, B

## Data preparation:

In [154]:
eee.head()

,Men's CT 2019 Ranking,Heat wins,Avg. heat score,Rookie year,Stance,First season,Age,Height (cm),Weight (kg),Country,...,Event 2: Points,Event 3: Points,Event 4: Points,Event 5: Points,Event 6: Points,Event 7: Points,Event 8: Points,Event 9: Points,Event 10: Points,Event 11: Points
Athlete,,,,,,,,,,,,,,,,,,,,,
Italo Ferreira,1,33,13.35,2015,Goofy,2009 Men's QS,25,168,68,Brazil,...,4745,1330,4745,1330,7800,1330,3320,7800,10000,10000
Gabriel Medina,2,31,13.64,2012,Goofy,2008 Men's QS,26,180,77,Brazil,...,4745,1330,1330,4745,10000,7800,10000,3320,3320,7800
Jordy Smith,3,25,12.1,2008,Regular,2006 Men's CT,31,190,88,South Africa,...,6085,1330,4745,7800,3320,6085,4745,3320,7800,1330
Filipe Toledo,4,27,13.62,2013,Regular,2009 Men's QS,24,175,70,Brazil,...,7800,4745,1330,10000,6085,3320,7800,1330,4745,1330
Kolohe Andino,5,28,11.98,2012,Regular,2008 Men's QS,25,180,79,United States,...,1330,4745,7800,6085,6085,1330,1330,4745,4745,3320


In [158]:
eee.tail()

,Men's CT 2019 Ranking,Heat wins,Avg. heat score,Rookie year,Stance,First season,Age,Height (cm),Weight (kg),Country,...,Event 2: Points,Event 3: Points,Event 4: Points,Event 5: Points,Event 6: Points,Event 7: Points,Event 8: Points,Event 9: Points,Event 10: Points,Event 11: Points
Athlete,,,,,,,,,,,,,,,,,,,,,
Jadson Andre,32,4,9.98,2010,Goofy,2008 Men's QS,29,170,70,Brazil,...,1330,1330,1330,1330,265,4745,1330,265,1330,1330
Ricardo Christie,123,1,11.17,3.56,Regular,2008 Men's QS,31,185,84,New Zealand,...,1330,1330,1330,1330,1330,1330,1330,265,265,3320
Frederico Morais,34,3,10.4,2.99,Regular,2008 Men's QS,28,185,81,Portugal,...,0,0,265,6085,1330,265,0,1330,1330,265
Adriano de Souza,35,2,11.03,2005,Regular,2006 Men's CT,32,170,66,Brazil,...,265,265,265,1330,1330,4745,265,265,265,265
Mikey Wright,36,3,10.66,2.74,Regular,2012 Men's JUN,23,183,80,Australia,...,1330,1330,265,265,265,265,265,265,265,265


In [153]:
eee.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, Italo Ferreira to Mikey Wright
Data columns (total 32 columns):
Men's CT 2019 Ranking    36 non-null object
Heat wins                36 non-null object
Avg. heat score          36 non-null object
Rookie year              36 non-null object
Stance                   36 non-null object
First season             36 non-null object
Age                      36 non-null object
Height (cm)              36 non-null object
Weight (kg)              36 non-null object
Country                  36 non-null object
Event 1: Place           36 non-null object
Event 2: Place           36 non-null object
Event 3: Place           36 non-null object
Event 4: Place           36 non-null object
Event 5: Place           36 non-null object
Event 6: Place           36 non-null object
Event 7: Place           36 non-null object
Event 8: Place           36 non-null object
Event 9: Place           36 non-null object
Event 10: Place          36 non-null object

In [162]:
eee.describe()

,Men's CT 2019 Ranking,Heat wins,Avg. heat score,Rookie year,Age,Height (cm),Weight (kg),Event 1: Points,Event 2: Points,Event 3: Points,Event 4: Points,Event 5: Points,Event 6: Points,Event 7: Points,Event 8: Points,Event 9: Points,Event 10: Points,Event 11: Points
count,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000
mean,18.500000,12.138889,11.378611,1062.444444,27.805556,178.388889,77.666667,2596.250000,2556.666667,2681.111111,2633.194444,2688.472222,2688.472222,2633.194444,2592.361111,2534.444444,2629.305556,2658.888889
std,10.535654,8.903138,1.026270,1016.528362,5.001825,6.634446,6.628941,2307.363794,2281.526744,2275.128289,2275.027206,2266.618570,2266.618570,2275.027206,2347.765042,2296.781530,2316.056424,2291.667238
min,1.000000,1.000000,9.730000,2.000000,21.000000,166.000000,66.000000,0.000000,0.000000,0.000000,265.000000,265.000000,265.000000,265.000000,0.000000,265.000000,265.000000,265.000000
25%,9.750000,3.750000,10.737500,3.000000,24.750000,174.500000,72.750000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000
50%,18.500000,11.000000,11.125000,1997.500000,27.000000,180.000000,77.500000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000
75%,27.250000,16.250000,11.982500,2012.000000,31.000000,183.000000,80.250000,3320.000000,3320.000000,3320.000000,3320.000000,3320.000000,3320.000000,3320.000000,3320.000000,3320.000000,3320.000000,3320.000000
max,36.000000,33.000000,13.640000,2016.000000,47.000000,190.000000,95.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000


In [163]:
eee.describe(include='object')

,Stance,First season,Country,Event 1: Place,Event 2: Place,Event 3: Place,Event 4: Place,Event 5: Place,Event 6: Place,Event 7: Place,Event 8: Place,Event 9: Place,Event 10: Place,Event 11: Place
count,36,36,36,36,36,36,36,36,36,36,36,36,36,36
unique,2,9,10,9,10,9,8,8,8,8,9,8,8,8
top,Regular,2008 Men's QS,Brazil,17th,17th,17th,17th,17th,17th,17th,17th,17th,17th,17th
freq,26,13,12,15,15,15,15,15,15,16,12,14,13,14


In [159]:
eee[['Avg. heat score']]=eee[['Avg. heat score']].astype(float)
eee[['Event {}: Points'.format(i) for i in range(1, 12)]]=eee[['Event {}: Points'.format(i) for i in range(1, 12)]].astype(int)
eee[["Men's CT 2019 Ranking", 'Heat wins', 'Rookie year', 'Age', 'Height (cm)', 'Weight (kg)']]=eee[["Men's CT 2019 Ranking", 'Heat wins', 'Rookie year', 'Age', 'Height (cm)', 'Weight (kg)']].astype(int)
eee[["Men's CT 2019 Ranking"]]=[i for i in range(1, 37)]

In [17]:
eee.head()

,Men's CT 2019 Ranking,Heat wins,Avg. heat score,Rookie year,Stance,First season,Age,Height (cm),Weight (kg),Country,...,Event 2: Points,Event 3: Points,Event 4: Points,Event 5: Points,Event 6: Points,Event 7: Points,Event 8: Points,Event 9: Points,Event 10: Points,Event 11: Points
Athlete,,,,,,,,,,,,,,,,,,,,,
Italo Ferreira,1,33,13.35,2015,Goofy,2009 Men's QS,25,168,68,Brazil,...,4745,1330,4745,1330,7800,1330,3320,7800,10000,10000
Gabriel Medina,2,31,13.64,2012,Goofy,2008 Men's QS,26,180,77,Brazil,...,4745,1330,1330,4745,10000,7800,10000,3320,3320,7800
Jordy Smith,3,25,12.10,2008,Regular,2006 Men's CT,31,190,88,South Africa,...,6085,1330,4745,7800,3320,6085,4745,3320,7800,1330
Filipe Toledo,4,27,13.62,2013,Regular,2009 Men's QS,24,175,70,Brazil,...,7800,4745,1330,10000,6085,3320,7800,1330,4745,1330
Kolohe Andino,5,28,11.98,2012,Regular,2008 Men's QS,25,180,79,United States,...,1330,4745,7800,6085,6085,1330,1330,4745,4745,3320


In [49]:
#eee.describe(include=['object'])
eee.describe(include='all')

,Men's CT 2019 Ranking,Heat wins,Avg. heat score,Rookie year,Stance,First season,Age,Height (cm),Weight (kg),Country,...,Event 2: Points,Event 3: Points,Event 4: Points,Event 5: Points,Event 6: Points,Event 7: Points,Event 8: Points,Event 9: Points,Event 10: Points,Event 11: Points
count,36.000000,36.000000,36.000000,36.000000,36,36,36.000000,36.000000,36.000000,36,...,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000
unique,NaN,NaN,NaN,NaN,2,9,NaN,NaN,NaN,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,Regular,2008 Men's QS,NaN,NaN,NaN,Brazil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,26,13,NaN,NaN,NaN,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,18.500000,12.138889,11.378611,1062.444444,NaN,NaN,27.805556,178.388889,77.666667,NaN,...,2556.666667,2681.111111,2633.194444,2688.472222,2688.472222,2633.194444,2592.361111,2534.444444,2629.305556,2658.888889
std,10.535654,8.903138,1.026270,1016.528362,NaN,NaN,5.001825,6.634446,6.628941,NaN,...,2281.526744,2275.128289,2275.027206,2266.618570,2266.618570,2275.027206,2347.765042,2296.781530,2316.056424,2291.667238
min,1.000000,1.000000,9.730000,2.000000,NaN,NaN,21.000000,166.000000,66.000000,NaN,...,0.000000,0.000000,265.000000,265.000000,265.000000,265.000000,0.000000,265.000000,265.000000,265.000000
25%,9.750000,3.750000,10.737500,3.000000,NaN,NaN,24.750000,174.500000,72.750000,NaN,...,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000
50%,18.500000,11.000000,11.125000,1997.500000,NaN,NaN,27.000000,180.000000,77.500000,NaN,...,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000,1330.000000
75%,27.250000,16.250000,11.982500,2012.000000,NaN,NaN,31.000000,183.000000,80.250000,NaN,...,3320.000000,3320.000000,3320.000000,3320.000000,3320.000000,3320.000000,3320.000000,3320.000000,3320.000000,3320.000000


In [34]:
eee.groupby('Country').count()

,Men's CT 2019 Ranking,Heat wins,Avg. heat score,Rookie year,Stance,First season,Age,Height (cm),Weight (kg),Event 1: Place,...,Event 2: Points,Event 3: Points,Event 4: Points,Event 5: Points,Event 6: Points,Event 7: Points,Event 8: Points,Event 9: Points,Event 10: Points,Event 11: Points
Country,,,,,,,,,,,,,,,,,,,,,
Australia,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
Brazil,12,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12
France,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
Hawaii,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
Italy,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Japan,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
New Zealand,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Portugal,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
South Africa,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [57]:
df = pd.DataFrame(fff['Country'].value_counts())
print(df)

print(df.loc[:,'Country'])
df.index



               Country
Brazil              12
Australia            8
United States        4
Hawaii               4
France               3
Portugal             1
South Africa         1
New Zealand          1
Japan                1
Italy                1
Brazil           12
Australia         8
United States     4
Hawaii            4
France            3
Portugal          1
South Africa      1
New Zealand       1
Japan             1
Italy             1
Name: Country, dtype: int64


Index(['Brazil', 'Australia', 'United States', 'Hawaii', 'France', 'Portugal',
       'South Africa', 'New Zealand', 'Japan', 'Italy'],
      dtype='object')

In [18]:
eee.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, Italo Ferreira to Mikey Wright
Data columns (total 32 columns):
Men's CT 2019 Ranking    36 non-null int64
Heat wins                36 non-null int32
Avg. heat score          36 non-null float64
Rookie year              36 non-null int32
Stance                   36 non-null object
First season             36 non-null object
Age                      36 non-null int32
Height (cm)              36 non-null int32
Weight (kg)              36 non-null int32
Country                  36 non-null object
Event 1: Place           36 non-null object
Event 2: Place           36 non-null object
Event 3: Place           36 non-null object
Event 4: Place           36 non-null object
Event 5: Place           36 non-null object
Event 6: Place           36 non-null object
Event 7: Place           36 non-null object
Event 8: Place           36 non-null object
Event 9: Place           36 non-null object
Event 10: Place          36 non-null object
Even

## Data Visualisation:

In [65]:
import plotly.express as px
import plotly.graph_objects as go

In [60]:
fig = px.scatter(fff, x='Country',y='Heat wins', color='Athlete', title='Heat wins by Country and Athlete')
fig.show()

In [58]:
fig = px.pie(df, values=df.loc[:,'Country'], names=df.index, title='Country counts')
fig.show()

In [138]:
popici = eee[['Event {}: Points'.format(i) for i in range(1, 12)]].T.cumsum().T

top10=popici[:10]
top10.columns=['Points after event {}'.format(i) for i in range(1, 12)]
top10 = top10.T


In [152]:
# Initialize figure
fig = go.Figure()

# Add Traces

fig.add_trace(
    go.Scatter(x=list(top10.index),
               y=list(top10.loc[:,'Italo Ferreira']),
               name="Italo Ferreira",
               line=dict(color="#f2a81a")))

fig.add_trace(
    go.Scatter(x=list(top10.index),
               y=list(top10.loc[:,'Gabriel Medina']),
               name="Gabriel Medina",
               line=dict(color="#ff0000")))

fig.add_trace(
    go.Scatter(x=list(top10.index),
               y=list(top10.loc[:,'Jordy Smith']),
               name="Jordy Smith",
               line=dict(color="#0051ff")))

fig.add_trace(
    go.Scatter(x=list(top10.index),
               y=list(top10.loc[:,'Filipe Toledo']),
               name="Filipe Toledo",
               line=dict(color="#1ddb00")))

fig.add_trace(
    go.Scatter(x=list(top10.index),
               y=list(top10.loc[:,'Kolohe Andino']),
               name="Kolohe Andino",
               line=dict(color="#db00d1")))

fig.add_trace(
    go.Scatter(x=list(top10.index),
               y=list(top10.loc[:,'Kanoa Igarashi']),
               name="Kanoa Igarashi",
               visible=False,
               line=dict(color="#00ffdf")))

fig.add_trace(
    go.Scatter(x=list(top10.index),
               y=list(top10.loc[:,'John John Florence']),
               name="John John Florence",
               visible=False,
               line=dict(color="#ff6700")))

fig.add_trace(
    go.Scatter(x=list(top10.index),
               y=list(top10.loc[:,'Kelly Slater']),
               name="Kelly Slater",
               visible=False,
               line=dict(color="#9bff85")))

fig.add_trace(
    go.Scatter(x=list(top10.index),
               y=list(top10.loc[:,'Owen Wright']),
               name="Owen Wright",
               visible=False,
               line=dict(color="#ff85f4")))

fig.add_trace(
    go.Scatter(x=list(top10.index),
               y=list(top10.loc[:,'Jeremy Flores']),
               name="Jeremy Flores",
               visible=False,
               line=dict(color="#9aa6ff")))

# Add Annotations and Buttons

fig.update_layout(
    updatemenus=[
        go.layout.Updatemenu(
            type="buttons",
            direction="right",
            active=0,
            x=0.65,
            y=1.2,
            buttons=list([
                dict(label="Top 5 athletes",
                     method="update",
                     args=[{"visible": [True, True, True, True, True, False, False, False, False, False]},
                           {"title": "Cummulative points Top 5",
                            "annotations": []}]),
                 dict(label="Top 10 athletes",
                     method="update",
                     args=[{"visible": [True, True, True, True, True, True, True, True, True, True]},
                           {"title": "Cummulative points Top 10",
                            "annotations": []}]),
            ]),
        )
    ])

# Set title
fig.update_layout(
    title_text="Cummulative points Top 5",
    xaxis_domain=[0.05, 1.0]
)

fig.show()